In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#  El Índice Oceánico de El Niño (ONI en inglés)

## Descargar los datos de internet

Página web: http://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php

Datos: https://www.esrl.noaa.gov/psd/data/correlation/oni.data

### Uso de pandas para descargar los datos de Internet

In [ ]:
oni = pd.read_csv("https://www.esrl.noaa.gov/psd/data/correlation/oni.data", skiprows=1, 
                  skipfooter=8, header=None, delim_whitespace=True, engine="python")
#oni.to_csv("./data<</oni.txt", index=False)
oni.head()

In case you do not have an internet connection the ONI data set is stored in the folder `./data/oni.txt`

In [ ]:
oni_raw = pd.read_csv("./data/oni.txt")
oni_raw.tail()

## Limpieza de datos

In [ ]:
oni_raw = oni_raw.replace(-99.90, np.nan)
oni_raw.tail()

## Formato ancho a largo

In [ ]:
oni_melted = pd.melt(oni_raw, id_vars="0", value_name="ONI", var_name="Month")
oni_melted.tail()

## Añadir columna de fecha

In [ ]:
oni_melted["Date"] = oni_melted.apply(lambda x: pd.to_datetime(str(x[0]) +"-"+ str(x["Month"])).to_period("M"), axis=1)
oni_melted.head()

## Subconjunto del conjunto de datos

In [ ]:
oni = oni_melted[["Date", "ONI"]].sort_values("Date").set_index("Date")
oni.head()

In [ ]:
oni = oni["2005":"2015"]
oni

## Trazar los datos

In [ ]:
fig, ax = plt.subplots(figsize=(13,4))
oni.plot(ax=ax, color="k")
ax.fill_between(oni.index, 0.5, oni.ONI, where=oni.ONI >= 0.5, color="red", label="El Nino", interpolate=True)
ax.fill_between(oni.index, -0.5, oni.ONI, where=oni.ONI <= -0.5, color="blue", label="La Nina", interpolate=True)
ax.grid()
ax.legend()
ax.set_ylim(-3,3)
ax.set_title("Ocean Nino Index (ONI)", size=14)
ax.axvline("2012-8", color="green", linestyle="dashed");

## Refactorización del código

In [ ]:
# %load src/oni_plot.py
def plot_oni(df, min_max = None, vline=None, ax=None):
    
    import pandas as pd
    import matplotlib.pyplot as plt
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(13,4))
    
    oni_melted = pd.melt(df, id_vars="0", value_name="ONI", var_name="Month")
    oni_melted["Date"] = oni_melted.apply(lambda x: pd.to_datetime(str(x[0]) +"-"+ str(x["Month"])).to_period("M"), axis=1)
    oni = oni_melted[["Date", "ONI"]].sort_values("Date").reset_index(drop=True).set_index("Date")
    if min_max is not None:
        oni = oni[str(min_max[0]):str(min_max[1])]        

    oni.plot(ax=ax, color="k")
    ax.fill_between(oni.index, 0.5, oni.ONI, where=oni.ONI >= 0.5,color="red", label="El Nino", interpolate=True)
    ax.fill_between(oni.index, -0.5, oni.ONI, where=oni.ONI <= -0.5,color="blue", label="La Nina", interpolate=True)
    ax.grid()
    ax.legend()
    ax.set_ylim(-3,3)
    ax.set_title("Ocean Nino Index (ONI)", size=14)
    
    if vline is not None:
        ax.axvline(str(vline), color="green", linestyle="dashed");
        

In [ ]:
oni = pd.read_csv("./data/oni.txt")
#oni

In [ ]:
oni = pd.read_csv("./data/oni.txt")
plot_oni(oni)

In [ ]:
plot_oni(oni, min_max = [2000, 2015])

In [ ]:
plot_oni(oni, min_max = [2000, 2015], vline="2005-1")